
# 필요한 라이브러리 설치 및 import 
1. Import and Install Dependencies

GPU를 사용하는 tensorflow와 mediapipe 설치 <br>

 mediapipe란 ? 주로 멀티 모델 오디오, 비디오 또는 모든 시계열 데이터를 구축하는 데 사용되는 프레임 워크입니다. <br>



In [2]:
# 필요한 라이브러리 설치 

#using tensorflow with GPU, mediapipe detacting movements
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

mp_holistic = mp.solutions.holistic

# MP Holistic을 사용하여 이미지의 좌표 출력하기 

 <br>
#mp.solutions.holistic 이란 ? -> 실시간 모션인식 (얼굴,몸의 분절을 실시간으로 추적한다.)

참고 사이트 <br>
https://google.github.io/mediapipe/solutions/holistic.html



mp_drawing = 점을 연결하여 선을 보여준다.

In [2]:
# 이미지의 좌표를 추적하는 함수 만들기 
#이미지 -> RGB형식으로 만들어 모델에 대입한 결과를 results에 담고, 원상태로 만든 이미지와 results를 출력한다. 

def mediapipe_detection(image, model):   # 대상을 추적한다. 
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction 모델에 대입 
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [54]:
print(results)
#<class 'mediapipe.python.solution_base.SolutionOutputs'>

results.face_landmarks.landmark
#x: 0.5144715905189514
#y: 0.24100039899349213
#z: -0.015107447281479836

#results -> <face, pose, left_hand, right_hand> 총 네 개의 결과값이 있다,
#하나의 값 (results.face_landmarks.landmark) 당 x,y,z 총 3개의 값이 3차원으로 존재한다. 

NameError: name 'results' is not defined

# 3. 위에서 얻은 results의 값을 numpy형태로 출력하기 
Extract Keypoint Values

results.pose_landmarks.landmark -> [res.x, res.y, res.z, res.visibility]으로 총 4개의 값으로 구성 <br>
그외는 <br>
results.face_landmarks.landmark -> [res.x, res.y, res.z]으로 구성 


In [3]:
# pose, face, lh, rh의 순서로 저장한다. 
# np.concatenate 를 하여 네개의 값을 하나의 넘파이 배열로 만든다.  
#만약 frame에 값이 출력되지 않았다면, (사진에서 왼손이 보이지 않아 left_hand의 result가 none으로 나올 경우) 0으로 채운다. 

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

#extract_keypoints(results).shape 
# (1662,) -> 33*4 + 468*3  + 21*3 + 21*3  

# 훈련데이터 동영상 넘파이 배열로 저장하기 
Convert Keypoint Values from videos

< 데이터 출처>  <br>
: Ai hub https://aihub.or.kr/keti_data_board/language_intelligence

<데이터 구성> <br>
데이터 크기 : 3000개의 수화 동영상 <br>
녹화 참여 인원수 : 총 10명 <br>
단어 종류 : 419개 

<데이터프레임 설명> <br>
dt : AI hub에서 가져온 동영상 파일 설명 엑셀파일 원본 <br>
dt1 : 문장과 단어가 모두 포함되어 있는 dt원본파일에서 단어 동영상만 추출 <br>
list_a : dt1의 한국어 컬럼을 단어의 개수 별로 정리한 데이터 프레임 <단어/횟수>로 구성.  

## 필요한 데이터프레임 생성 

In [ ]:
#tensorflow에서 엑셀파일 여는 라이브러리 다운 

#!pip install pandas
!pip install openpyxl

In [406]:
#dt 데이터프레임 생성 
import pandas as pd
dt=pd.read_excel("C://Users//User//sign//단어list10.15.xlsx")
dt
dt.head()

,번호,언어 제공자 ID,취득연도,방향,타입(단어/문장),파일명,한국어
0,1.0,1.0,2017.0,정면,단어,KETI_SL_0000000001.MOV,0.0
1,2.0,1.0,2017.0,정면,단어,KETI_SL_0000000002.MOV,1.0
2,3.0,1.0,2017.0,정면,단어,KETI_SL_0000000003.MOV,2.0
3,4.0,1.0,2017.0,정면,단어,KETI_SL_0000000004.MOV,3.0
4,5.0,1.0,2017.0,정면,단어,KETI_SL_0000000005.MOV,4.0


In [407]:
# dt 한국어컬럼 중 float 형 int형으로 바꾸기 

for i in range(len(dt['한국어'])):
    try:
        dt['한국어'][i] = int(dt['한국어'][i])
        dt['한국어'][i] = str(dt['한국어'][i])
    except:
        dt['한국어'][i] == dt['한국어'][i]

C:\Users\solag\AppData\Local\Temp/ipykernel_18316/2159450155.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt['한국어'][i] = int(dt['한국어'][i])
C:\Users\solag\AppData\Local\Temp/ipykernel_18316/2159450155.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt['한국어'][i] = str(dt['한국어'][i])


In [408]:
#파일명 마지막 세글자.avi로 바꾸기 
dt['파일명'] = dt['파일명'].apply(lambda x:x[:-3]) + dt['파일명'].apply(lambda x:x[-3:]).replace('MOV','avi')
dt['파일명'] = dt['파일명'].apply(lambda x:x[:-3]) + dt['파일명'].apply(lambda x:x[-3:]).replace('MTS','avi')
dt = dt.sort_values('파일명')
dt

,번호,언어 제공자 ID,취득연도,방향,타입(단어/문장),파일명,한국어
0,1.0,1.0,2017.0,정면,단어,KETI_SL_0000000001.avi,0
1,2.0,1.0,2017.0,정면,단어,KETI_SL_0000000002.avi,1
2,3.0,1.0,2017.0,정면,단어,KETI_SL_0000000003.avi,2
3,4.0,1.0,2017.0,정면,단어,KETI_SL_0000000004.avi,3
4,5.0,1.0,2017.0,정면,단어,KETI_SL_0000000005.avi,4
...,...,...,...,...,...,...,...
2988,6779.0,9.0,2017.0,정면,단어,KETI_SL_0000006779.avi,74
2989,6780.0,9.0,2017.0,정면,단어,KETI_SL_0000006780.avi,75
2990,6781.0,9.0,2017.0,정면,단어,KETI_SL_0000006781.avi,76
2991,6782.0,9.0,2017.0,정면,단어,KETI_SL_0000006782.avi,77


In [409]:
## dt1 데이터 프레임 생성 
#전체 데이터 단어만 선택한 후 3000개 데이터 선택 

import pandas as pd
dt1=dt[dt['타입(단어/문장)']=='단어']
dt1 = dt1[:6783]
dt1 = dt1[dt1['한국어'].isin(['배','화장실','동생','학생','내년','앞'])]


In [410]:
#dt1.to_excel('dt1.xlsx')
a = [i for i in range(0,42)]
dt1['번호']=a
dt1.set_index('번호',inplace = True)
dt1

,언어 제공자 ID,취득연도,방향,타입(단어/문장),파일명,한국어
번호,,,,,,
0,1.0,2017.0,정면,단어,KETI_SL_0000000167.avi,내년
1,1.0,2017.0,정면,단어,KETI_SL_0000000190.avi,동생
2,1.0,2017.0,정면,단어,KETI_SL_0000000225.avi,배
3,1.0,2017.0,정면,단어,KETI_SL_0000000287.avi,앞
4,1.0,2017.0,정면,단어,KETI_SL_0000000399.avi,학생
5,1.0,2017.0,정면,단어,KETI_SL_0000000418.avi,화장실
6,3.0,2017.0,정면,단어,KETI_SL_0000001749.avi,내년
7,3.0,2017.0,정면,단어,KETI_SL_0000001774.avi,동생
8,3.0,2017.0,정면,단어,KETI_SL_0000001810.avi,배


In [411]:
# 단어별로 동영상이 몇게 있는지 출력하는 list_a 데이터 프레임 생성 

#한국어컬럼만 추출
aa = dt1[['한국어']].copy()

#한국어 컬럼에 대한 개수 추출
list_a =aa['한국어'].value_counts()

#인덱스를 없애서 총 2개의 컬럼을 가진 데이터 프레임 생성 [한국어.횟수]
list_a =list_a.reset_index()

# 데이터프레임 컬렴명 변겅 
list_a.columns = ['한국어','횟수']
list_a = list_a.sort_values('한국어')
list_a

#len(list_a)
#419

,한국어,횟수
0,내년,7
1,동생,7
2,배,7
3,앞,7
4,학생,7
5,화장실,7


## numpy형태로 변환한 파일 담을 폴더 생성 

<폴더 구조> <br>
train 폴더 <br>
단어의 뜻을 이름으로 하는 폴더 419개  <br>
각각의 단어 폴더 안에 단어 당 촬영한 동영상 개수 만큼 폴더 생성   <br>
각각의 동영상 개수 폴더 안에 프레임 횟수만큼 폴더를 만들고 하나의 폴더안에 프레임 하나의 넘파이 배열이 담긴 파이썬 파일 첨부 

In [422]:
# 단어별 -동영상개수만큼의 폴더 생성하기 
import os
# train 폴더 생성
os.mkdir( 'C://Users//User//sign//train')

# 단어별 폴더 생성 
for k in range(6): 
    a = 'C://Users//User//sign//train//' + str(list_a['한국어'][k])  # 동영상 이름 
    os.mkdir(a)
    # 단어 폴더 안에 동영상의 횟수만큼 폴더 생성 
    for i in range(list_a['횟수'][k]):
        b= 'C://Users//User//sign//train//' + str(list_a['한국어'][k]) + '//' + str(i)
        os.mkdir(b)

## video3000에 있는 동영상 읽어와서 numpy배열로 변환한 후 위에서 만든 train 폴더에 저장 


In [423]:
#폴더의 개수 출력하는 함수 생성 
def dir_empty(dir_path):
    return len(os.listdir(dir_path))


# holistic 모델 사용 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # 3000개의 동영상 분석 
    for k in range(42):
        # dt1['파일명']의 동영상 3000개 분석 
        #dt1 -> 단어별로 정리한 동영상 파일 3000개 
        
        videos = "C://Users//User//sign//video3000//" + dt1['파일명'][k] 
        print(videos)

        
        #openCV를 열고 동영상을 읽어들인다. -> 동영상의 처음부터 끝까지 frame형태로 기록된다. 
        # -> 출력물 ret. frame 
        
        # ret이란 frame이 존재하는지 존재하지 않는지 True, False로 출력한다. 
        # 동영상이 끝나 더이상의 frame이 존재하지 않으면 False출력 
        
        cam = cv2.VideoCapture(videos)
        
        #출력하는 동영상의 해당 단어 kor에 담기 
        kor = dt1['한국어'][k]
        
        # videocount -> 해당 단어의 몇번째 동영상인지 출력 
        videocount =0

      # 
        while (True):
            #만약 해당 단어 폴더의 vidoecount의 이름을 가진 폴더의 길이가 0이라면(비어있다면)
            #  해당단어 - video count - frame 개수 별로 만든 폴더에 넘파이 파일 저장 
            if dir_empty('C://Users//User//sign//train//' + kor + '//'+str(videocount)) == 0:
                
                #프레임 개수를 알려주는 currentframe 
                currentframe = 0

                while (True):
                    # 여기서의 True 가 밑의 ret == True로 연결
                    #ret 이 True 라면 계속 반복, 즉 프레임이 남아있다면 계속해서 읽어들인다. 



                    # 위에서 읽은 cam을 frame별로 하나씩 읽어들이겠다.
                    ret,frame = cam.read()


                    # 만약 프레임이 남아있다면 
                    if ret == True:
                        
                        image, results = mediapipe_detection(frame, holistic)  # frame에 있는 점들을 추적하여 result에 넣는다. 
                        keypoints = extract_keypoints(results) #result를 하나의 넘파이 배열로 만든다
                        
                        # 프레임 번호를 폴더명으로 하는 폴더 생성 
                        os.mkdir('C://Users//User//sign//train//' + kor + '//'+str(videocount)+'//'+ str(currentframe))
                        
                        #위에서 만든 폴더에 넘파이 배열 파일을 넣겠다. 
                        npy_path = 'C://Users//User//sign//train//' + kor + '//'+str(videocount)+'//'+ str(currentframe)+'//' +str(currentframe)
                        np.save(npy_path, keypoints)
                        
                        #다음프레임으로 이동 
                        currentframe +=1
                        


                    #만약 프레임이 끝났다면 즉, 하나의 동영상이 완료되었다면 종료 (두번째 while 문 종료)
                    else:
                        break
                
                # 하나의 동영상 추출 완료 ( if문 종료 )
                # dir_empty('C://Users//User//sign//train//' + kor + '//'+str(videocount)) == 0: 종료 
                # 완료를 알려주는 문장 출력 
                print(kor + '단어' + str(currentframe) + '프레임 추출 완료======================================')
                
                break
                
    #첫번째 while 문 
    #만약 ir_empty('C://Users//User//sign//train//' + kor + '//'+str(videocount)) == 0 이 아니라면, 
    # 즉 단어 - 동영상 횟수를 나타내는 폴더가 비어있지 않다면 다음폴더로 이동 
    #ex ) 사자- 1번째 동영상의 폴더의 길이가 0이아니라면 사자-2 번째 폴더로 이동하고 위의 문장을 다시 실행 
    
            else:
                videocount +=1
                continue

C://Users//User//sign//video3000//KETI_SL_0000000167.avi
내년단어56프레임 추출 완료======================================
C://Users//User//sign//video3000//KETI_SL_0000000190.avi
동생단어131프레임 추출 완료======================================
C://Users//User//sign//video3000//KETI_SL_0000000225.avi
배단어139프레임 추출 완료======================================
C://Users//User//sign//video3000//KETI_SL_0000000287.avi
앞단어128프레임 추출 완료======================================
C://Users//User//sign//video3000//KETI_SL_0000000399.avi
학생단어65프레임 추출 완료======================================
C://Users//User//sign//video3000//KETI_SL_0000000418.avi
화장실단어116프레임 추출 완료======================================
C://Users//User//sign//video3000//KETI_SL_0000001749.avi
내년단어52프레임 추출 완료======================================
C://Users//User//sign//video3000//KETI_SL_0000001774.avi
동생단어59프레임 추출 완료======================================
C://Users//User//sign//video3000//KETI_SL_0000001810.avi
배단어55프레임 추출 완료======================================

In [424]:
import os
import shutil
file_list_1=os.listdir("C://Users//User//sign//train")

In [431]:
##201로 맞추기
for i in range(len(file_list_1)):
    path="C://Users//USer//sign//train//" + str(file_list_1[i])
    file_list_2=os.listdir(path)
    for p in range(len(file_list_2)):
        path2=path+'//'+str(file_list_2[p])
        file_list_3=os.listdir(path2)
        #print(path2, len(file_list_3))
        path3=path2+'//'+str(len(file_list_3)-1)
        
        num=len(file_list_3)-1
        
        if num < 201 :
            
            while (True):
                if num < 201:
                    num=num+1
                    path4=path2+'//'+str(num)
                    print(path4)

                    if num < 201:
                        shutil.copytree(path3, path4)
                        #print(path4)

                        continue

                    #print(path4)
                    break
                
                #shutil.copytree("C://Users//User//sign//test//0", "C://Users//User//sign//test//0_2")
        
        
#print(max(count_list3))     0~431   432개


#len(file_list_3)

C://Users//USer//sign//train//내년//0//200
C://Users//USer//sign//train//내년//0//201
C://Users//USer//sign//train//내년//1//200
C://Users//USer//sign//train//내년//1//201
C://Users//USer//sign//train//내년//2//200
C://Users//USer//sign//train//내년//2//201
C://Users//USer//sign//train//내년//3//200
C://Users//USer//sign//train//내년//3//201
C://Users//USer//sign//train//내년//4//200
C://Users//USer//sign//train//내년//4//201
C://Users//USer//sign//train//내년//5//200
C://Users//USer//sign//train//내년//5//201
C://Users//USer//sign//train//내년//6//200
C://Users//USer//sign//train//내년//6//201
C://Users//USer//sign//train//동생//0//200
C://Users//USer//sign//train//동생//0//201
C://Users//USer//sign//train//동생//1//200
C://Users//USer//sign//train//동생//1//201
C://Users//USer//sign//train//동생//2//200
C://Users//USer//sign//train//동생//2//201
C://Users//USer//sign//train//동생//3//200
C://Users//USer//sign//train//동생//3//201
C://Users//USer//sign//train//동생//4//200
C://Users//USer//sign//train//동생//4//201
C://Users//USer/

In [ ]:
## 3차원 형태의 넘파이 배열을 모델에 넣기 위해서는 모든 프레임의 개수가 같아야 한다.
# 프레임의 개수가 가장 많은 영상을 찾아 그 영상에 맞춰 모든 영상의 마지막 프레임을 해당 프레임의 개수만큼 복사한다. 

# 6. 데이터 라벨링하기 


### 동영상 데이터 라벨링하기


In [7]:
!pip uninstall protobuf
#pip install --no-binary=protobuf protobuf

^C


In [3]:
# 다른 액션에 대한 레이블 만들기 
# 필요한 데이터 임포트
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [432]:
# 정답데이터 라벨링 하기 
label_map = {label:num for num, label in enumerate(list_a['한국어'])} # enumerate-> 0 부터 순서대로 나타내 주는 것 

In [433]:
label_map

{'내년': 0, '동생': 1, '배': 2, '앞': 3, '학생': 4, '화장실': 5}

In [2]:
actions = ['x-ray사진','고치다','고혈압','멈추다','수술','약','저혈압','춥다','충격','피곤하다']
label_index={label:num for num,label in enumerate(actions)}
label_index

{'x-ray사진': 0,
 '고치다': 1,
 '고혈압': 2,
 '멈추다': 3,
 '수술': 4,
 '약': 5,
 '저혈압': 6,
 '춥다': 7,
 '충격': 8,
 '피곤하다': 9}

In [128]:
##### 프레임 개수만 맞추면 됩니다. 
#!pip install tqdm 

In [4]:
from tqdm import trange, notebook

In [438]:
# 파일 불러와서 해당 영상에 대한 정답 데이터 labels에 담기 
# labeling actions to labels 

sequences, labels = [], []
DATA_PATH  =  'C:\\Users\\User\\sign\\train\\'
for kor in notebook.tqdm(list_a['한국어']):
    for videocount in np.array(os.listdir(os.path.join(DATA_PATH, kor))).astype(int): #단어 안에 있는 폴더의 이름을 리스트로 출력하고 int형으로 변경 
        window = []
        for currentframe in range(201):
            # 데이터 파이썬 파일로 불러오기 
            file = os.listdir(DATA_PATH + kor + '\\'+str(videocount)+ "\\" + str(currentframe) + '\\')[0]
            res = np.load(os.path.join(DATA_PATH, kor, str(videocount),str(currentframe),file))
            # 리스트에 append 하기 
            window.append(res)
            #sequence라는 리스트안에 window가 리스트 형터로 존재 
            #리스트 안의 리스트 
        sequences.append(window)
        labels.append(label_map[kor])

  0%|          | 0/6 [00:00<?, ?it/s]

In [5]:
sequences, labels = [], []
DATA_PATH  =  'C:\\Users\\solag\\sh_Data\\'
for action in  notebook.tqdm(actions):
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(40):
            # 데이터 파이썬 파일로 불러오기 
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            # 리스트에 append 하기 
            window.append(res)
        sequences.append(window)
        labels.append(label_index[action])

  0%|          | 0/10 [00:00<?, ?it/s]

[]


In [8]:
np.array(sequences).shape  #(600, 40, 1662)
# 총 90개의 비디오, 1편 당 30시퀀스 /  # (1662,)키포인트 -> 468*3 + 33*4 + 21*3 + 21*3 각(xyz 즉 세개씩 곱한다.)
#(42, 432, 1662)

(600, 40, 1662)

In [9]:
np.array(labels).shape  #(42,) 라벨링한 정답 데이터  #210

(600,)

In [10]:
X = np.array(sequences)

In [659]:
X.shape
#90 channels, 30 frames, 1662 keypoints (68*3 + 33*4 + 21*3 + 21*3)

#(42, 432, 1662)

(600, 40, 1662)

In [11]:
# 원핫인코딩 하기 
# one hot encoding 
y = to_categorical(labels).astype(int)

In [617]:
print(labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 

In [12]:
# 데이터 나누기
# split data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05) # 5%가 테스트 되겠다

In [13]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(570, 40, 1662)
(30, 40, 1662)
(570, 10)
(30, 10)


# LSTM Neural Network 만들기 

Build and Train LSTM Neural Network

In [ ]:
# 모델 생성 -> cnn 넣었을 때, 넣지 않았을 때 모두 해보기 

In [14]:
# 모델 임포트 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout,BatchNormalization  #LSTM layer 를  쓰겠다.  
from tensorflow.keras.callbacks import TensorBoard

In [15]:
# set call back ( prevent over fitting )
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
# 이 타입 쓰는 이유 CNN 과 비교 시 장점 
# 미디아 type + LSTM 레이어
# 1. Less DaTa Required 
# 2. Faster To train
# 3. Faster Detections 

In [529]:
print(list_a.shape[0])

6


In [634]:
model = Sequential()  # 시작 
# 30, 1662) 각각의 비디오 30 시퀀스 1662 키포인트



model.add(LSTM(84, return_sequences=True, activation='tanh', input_shape=(40,1662))) #64 유닛 -> LSTM은 다음 레이어에 sequences가 필요해서 다음 레이어가 있으면 True를 해준다
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(LSTM(64, return_sequences=False, activation='tanh'))# 다음은 dense슬라이드라 sequences 필요없어

model.add(Dense(64, kernel_regularizer='l2',activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(50, kernel_regularizer='l2',activation = 'sigmoid'))
model.add(BatchNormalization())
model.add(Dropout(0.3))


model.add(Dense(len(actions),kernel_regularizer='l2', activation='softmax')) # ctions.shape[0] -> 3네트워크 유닛 

In [531]:
print(len(actions))

7


In [607]:
model=Sequential()
model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(30,1662)))
model.add(LSTM(128,return_sequences=True,activation='relu'))
model.add(LSTM(64,return_sequences=False,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(len(actions),activation='softmax'))

In [16]:
model = Sequential()  

model.add(LSTM(20, return_sequences=True, activation='tanh', input_shape=(40,1662))) #LSTM은 다음 레이어에 sequences가 필요해서 다음 레이어가 있으면 True를 해준다
#model.add(LSTM(64, return_sequences=True, activation='tanh'))
model.add(LSTM(20, return_sequences=False, activation='tanh'))# 다음은 dense슬라이드라 sequences 필요없어

model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(20, activation='relu'))

model.add(BatchNormalization())
model.add(Dropout(0.2))


model.add(Dense(len(actions), activation='softmax')) # ctions.shape[0] -> 3네트워크 유닛 

# 아담 그레이드 , 비아너리 -> 바이너리 크로스 / 이건 분류하는 거-> multi class categoty  예측하는 확률을 보여준다. 
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.fit(X_train, y_train, epochs=2000,batch_size = 300, validation_data = (X_test, y_test),callbacks=[tb_callback])

Epoch 1/2000
2/2 [==============================] - 3s 1s/step - loss: 3.0547 - categorical_accuracy: 0.1050 - val_loss: 2.3215 - val_categorical_accuracy: 0.0667
Epoch 2/2000
2/2 [==============================] - 0s 169ms/step - loss: 2.2631 - categorical_accuracy: 0.2329 - val_loss: 2.2698 - val_categorical_accuracy: 0.0667
Epoch 3/2000
2/2 [==============================] - 0s 169ms/step - loss: 1.9945 - categorical_accuracy: 0.2992 - val_loss: 2.2619 - val_categorical_accuracy: 0.0667
Epoch 4/2000
2/2 [==============================] - 0s 167ms/step - loss: 1.8336 - categorical_accuracy: 0.3095 - val_loss: 2.2888 - val_categorical_accuracy: 0.0667
Epoch 5/2000
2/2 [==============================] - 0s 167ms/step - loss: 1.7458 - categorical_accuracy: 0.3689 - val_loss: 2.2780 - val_categorical_accuracy: 0.0667
Epoch 6/2000
2/2 [==============================] - 0s 173ms/step - loss: 1.6155 - categorical_accuracy: 0.4585 - val_loss: 2.2976 - val_categorical_accuracy: 0.0667
Epoch 7

2/2 [==============================] - 1s 316ms/step - loss: 0.6007 - categorical_accuracy: 0.8678 - val_loss: 2.2913 - val_categorical_accuracy: 0.1333
Epoch 100/2000
2/2 [==============================] - 1s 258ms/step - loss: 0.6550 - categorical_accuracy: 0.8667 - val_loss: 2.2638 - val_categorical_accuracy: 0.2000
Epoch 101/2000
2/2 [==============================] - 0s 256ms/step - loss: 0.6657 - categorical_accuracy: 0.8451 - val_loss: 2.2056 - val_categorical_accuracy: 0.2333
Epoch 102/2000
2/2 [==============================] - 0s 177ms/step - loss: 0.6282 - categorical_accuracy: 0.8814 - val_loss: 2.2320 - val_categorical_accuracy: 0.2333
Epoch 103/2000
2/2 [==============================] - 0s 166ms/step - loss: 0.5927 - categorical_accuracy: 0.8723 - val_loss: 2.1622 - val_categorical_accuracy: 0.2667
Epoch 104/2000
2/2 [==============================] - 0s 171ms/step - loss: 0.5877 - categorical_accuracy: 0.8827 - val_loss: 2.2474 - val_categorical_accuracy: 0.2667
Epoch 1

Epoch 148/2000
2/2 [==============================] - 0s 214ms/step - loss: 0.5042 - categorical_accuracy: 0.9000 - val_loss: 2.0551 - val_categorical_accuracy: 0.2667
Epoch 149/2000
2/2 [==============================] - 0s 243ms/step - loss: 0.4511 - categorical_accuracy: 0.9240 - val_loss: 2.0328 - val_categorical_accuracy: 0.3000
Epoch 150/2000
2/2 [==============================] - 0s 188ms/step - loss: 0.4628 - categorical_accuracy: 0.9105 - val_loss: 2.0958 - val_categorical_accuracy: 0.3000
Epoch 151/2000
2/2 [==============================] - 0s 181ms/step - loss: 0.4726 - categorical_accuracy: 0.9205 - val_loss: 1.9437 - val_categorical_accuracy: 0.2000
Epoch 152/2000
2/2 [==============================] - 0s 187ms/step - loss: 0.4792 - categorical_accuracy: 0.9069 - val_loss: 2.0126 - val_categorical_accuracy: 0.1667
Epoch 153/2000
2/2 [==============================] - 0s 254ms/step - loss: 0.5486 - categorical_accuracy: 0.8782 - val_loss: 1.8943 - val_categorical_accuracy:

Epoch 197/2000
2/2 [==============================] - 0s 187ms/step - loss: 0.3676 - categorical_accuracy: 0.9368 - val_loss: 1.6753 - val_categorical_accuracy: 0.2333
Epoch 198/2000
2/2 [==============================] - 0s 190ms/step - loss: 0.3758 - categorical_accuracy: 0.9437 - val_loss: 1.6199 - val_categorical_accuracy: 0.4333
Epoch 199/2000
2/2 [==============================] - 0s 190ms/step - loss: 0.3932 - categorical_accuracy: 0.9253 - val_loss: 1.6966 - val_categorical_accuracy: 0.4667
Epoch 200/2000
2/2 [==============================] - 0s 172ms/step - loss: 0.4171 - categorical_accuracy: 0.9220 - val_loss: 1.7044 - val_categorical_accuracy: 0.4000
Epoch 201/2000
2/2 [==============================] - 0s 182ms/step - loss: 0.3969 - categorical_accuracy: 0.9335 - val_loss: 1.4723 - val_categorical_accuracy: 0.5667
Epoch 202/2000
2/2 [==============================] - 0s 192ms/step - loss: 0.3812 - categorical_accuracy: 0.9389 - val_loss: 1.4758 - val_categorical_accuracy:

Epoch 246/2000
2/2 [==============================] - 0s 184ms/step - loss: 0.3105 - categorical_accuracy: 0.9484 - val_loss: 1.4019 - val_categorical_accuracy: 0.4333
Epoch 247/2000
2/2 [==============================] - 0s 188ms/step - loss: 0.3344 - categorical_accuracy: 0.9404 - val_loss: 1.4334 - val_categorical_accuracy: 0.5000
Epoch 248/2000
2/2 [==============================] - 0s 184ms/step - loss: 0.3049 - categorical_accuracy: 0.9553 - val_loss: 2.0645 - val_categorical_accuracy: 0.2000
Epoch 249/2000
2/2 [==============================] - 0s 183ms/step - loss: 0.3117 - categorical_accuracy: 0.9413 - val_loss: 1.0678 - val_categorical_accuracy: 0.8000
Epoch 250/2000
2/2 [==============================] - 0s 173ms/step - loss: 0.2998 - categorical_accuracy: 0.9529 - val_loss: 1.1917 - val_categorical_accuracy: 0.6667
Epoch 251/2000
2/2 [==============================] - 0s 186ms/step - loss: 0.3373 - categorical_accuracy: 0.9561 - val_loss: 1.3167 - val_categorical_accuracy:

...

2/2 [==============================] - 0s 185ms/step - loss: 0.0518 - categorical_accuracy: 0.9977 - val_loss: 1.5009 - val_categorical_accuracy: 0.7667
Epoch 1557/2000
2/2 [==============================] - 0s 171ms/step - loss: 0.0521 - categorical_accuracy: 0.9954 - val_loss: 2.6345 - val_categorical_accuracy: 0.5333
Epoch 1558/2000
2/2 [==============================] - 0s 190ms/step - loss: 0.0566 - categorical_accuracy: 0.9931 - val_loss: 2.2364 - val_categorical_accuracy: 0.5667
Epoch 1559/2000
2/2 [==============================] - 0s 181ms/step - loss: 0.0533 - categorical_accuracy: 0.9919 - val_loss: 1.3888 - val_categorical_accuracy: 0.7667
Epoch 1560/2000
2/2 [==============================] - 0s 184ms/step - loss: 0.0583 - categorical_accuracy: 0.9931 - val_loss: 1.0190 - val_categorical_accuracy: 0.8333
Epoch 1561/2000
2/2 [==============================] - 0s 169ms/step - loss: 0.0521 - categorical_accuracy: 0.9977 - val_loss: 1.6325 - val_categorical_accuracy: 0.6000
Ep

2/2 [==============================] - 0s 153ms/step - loss: 0.0351 - categorical_accuracy: 0.9988 - val_loss: 0.0846 - val_categorical_accuracy: 1.0000
Epoch 1653/2000
2/2 [==============================] - 0s 156ms/step - loss: 0.0436 - categorical_accuracy: 0.9954 - val_loss: 0.1621 - val_categorical_accuracy: 0.9667
Epoch 1654/2000
2/2 [==============================] - 0s 143ms/step - loss: 0.0450 - categorical_accuracy: 0.9931 - val_loss: 0.0635 - val_categorical_accuracy: 1.0000
Epoch 1655/2000
2/2 [==============================] - 0s 159ms/step - loss: 0.0391 - categorical_accuracy: 0.9977 - val_loss: 0.1216 - val_categorical_accuracy: 0.9667
Epoch 1656/2000
2/2 [==============================] - 0s 153ms/step - loss: 0.0495 - categorical_accuracy: 0.9988 - val_loss: 0.2536 - val_categorical_accuracy: 0.9333
Epoch 1657/2000
2/2 [==============================] - 0s 155ms/step - loss: 0.0362 - categorical_accuracy: 0.9965 - val_loss: 0.1081 - val_categorical_accuracy: 0.9667
Ep

2/2 [==============================] - 0s 185ms/step - loss: 0.0467 - categorical_accuracy: 0.9965 - val_loss: 0.3323 - val_categorical_accuracy: 0.9000
Epoch 1749/2000
2/2 [==============================] - 0s 191ms/step - loss: 0.0304 - categorical_accuracy: 1.0000 - val_loss: 0.1465 - val_categorical_accuracy: 0.9333
Epoch 1750/2000
2/2 [==============================] - 0s 185ms/step - loss: 0.0365 - categorical_accuracy: 0.9977 - val_loss: 0.0403 - val_categorical_accuracy: 1.0000
Epoch 1751/2000
2/2 [==============================] - 0s 193ms/step - loss: 0.0382 - categorical_accuracy: 0.9988 - val_loss: 0.0599 - val_categorical_accuracy: 1.0000
Epoch 1752/2000
2/2 [==============================] - 0s 193ms/step - loss: 0.0313 - categorical_accuracy: 1.0000 - val_loss: 0.1678 - val_categorical_accuracy: 0.9000
Epoch 1753/2000
2/2 [==============================] - 0s 183ms/step - loss: 0.0312 - categorical_accuracy: 0.9965 - val_loss: 0.0813 - val_categorical_accuracy: 1.0000
Ep

2/2 [==============================] - 0s 200ms/step - loss: 0.0282 - categorical_accuracy: 1.0000 - val_loss: 0.0519 - val_categorical_accuracy: 1.0000
Epoch 1845/2000
2/2 [==============================] - 0s 222ms/step - loss: 0.0366 - categorical_accuracy: 0.9988 - val_loss: 0.0717 - val_categorical_accuracy: 1.0000
Epoch 1846/2000
2/2 [==============================] - 1s 281ms/step - loss: 0.0377 - categorical_accuracy: 0.9920 - val_loss: 0.0523 - val_categorical_accuracy: 1.0000
Epoch 1847/2000
2/2 [==============================] - 0s 254ms/step - loss: 0.0288 - categorical_accuracy: 0.9965 - val_loss: 0.1015 - val_categorical_accuracy: 0.9667
Epoch 1848/2000
2/2 [==============================] - 0s 278ms/step - loss: 0.0394 - categorical_accuracy: 0.9977 - val_loss: 0.1139 - val_categorical_accuracy: 0.9667
Epoch 1849/2000
2/2 [==============================] - 0s 237ms/step - loss: 0.0284 - categorical_accuracy: 1.0000 - val_loss: 0.0676 - val_categorical_accuracy: 1.0000
Ep

2/2 [==============================] - 0s 186ms/step - loss: 0.0316 - categorical_accuracy: 0.9954 - val_loss: 1.1053 - val_categorical_accuracy: 0.5000
Epoch 1941/2000
2/2 [==============================] - 0s 190ms/step - loss: 0.0318 - categorical_accuracy: 1.0000 - val_loss: 0.5193 - val_categorical_accuracy: 0.8000
Epoch 1942/2000
2/2 [==============================] - 0s 171ms/step - loss: 0.0365 - categorical_accuracy: 0.9977 - val_loss: 0.3428 - val_categorical_accuracy: 0.8667
Epoch 1943/2000
2/2 [==============================] - 0s 182ms/step - loss: 0.0337 - categorical_accuracy: 0.9965 - val_loss: 0.4538 - val_categorical_accuracy: 0.8667
Epoch 1944/2000
2/2 [==============================] - 0s 177ms/step - loss: 0.0345 - categorical_accuracy: 1.0000 - val_loss: 0.3924 - val_categorical_accuracy: 0.8667
Epoch 1945/2000
2/2 [==============================] - 0s 188ms/step - loss: 0.0339 - categorical_accuracy: 0.9977 - val_loss: 0.3469 - val_categorical_accuracy: 0.8333
Ep

In [623]:
# 아담 그레이드 , 비아너리 -> 바이너리 크로스 / 이건 분류하는 거-> multi class categoty  예측하는 확률을 보여준다. 
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [624]:
from tensorflow.keras.callbacks import EarlyStopping
callbacks=[EarlyStopping(monitor='categorical_accuracy',patience=10,verbose=1)]

In [625]:
model.fit(X_train, y_train, epochs=200, batch_size=30,validation_data(X_test,y_test),callbacks=[tb_callback])

Epoch 1/200
2/2 [==============================] - 5s 2s/step - loss: 4.4893 - categorical_accuracy: 0.1128 - val_loss: 3.7665 - val_categorical_accuracy: 0.2333
Epoch 2/200
2/2 [==============================] - 2s 749ms/step - loss: 3.5697 - categorical_accuracy: 0.2935 - val_loss: 3.7621 - val_categorical_accuracy: 0.2333
Epoch 3/200
2/2 [==============================] - 2s 743ms/step - loss: 3.1689 - categorical_accuracy: 0.3486 - val_loss: 3.7520 - val_categorical_accuracy: 0.2333
Epoch 4/200
2/2 [==============================] - 2s 740ms/step - loss: 2.9606 - categorical_accuracy: 0.4093 - val_loss: 3.7463 - val_categorical_accuracy: 0.2333
Epoch 5/200
2/2 [==============================] - 2s 736ms/step - loss: 2.8838 - categorical_accuracy: 0.4372 - val_loss: 3.7391 - val_categorical_accuracy: 0.2333
Epoch 6/200
2/2 [==============================] - 2s 737ms/step - loss: 2.7272 - categorical_accuracy: 0.4966 - val_loss: 3.7298 - val_categorical_accuracy: 0.2333
Epoch 7/200
2

2/2 [==============================] - 2s 920ms/step - loss: 0.7179 - categorical_accuracy: 0.9793 - val_loss: 2.6690 - val_categorical_accuracy: 0.3333
Epoch 100/200
2/2 [==============================] - 2s 905ms/step - loss: 0.6704 - categorical_accuracy: 0.9942 - val_loss: 2.6964 - val_categorical_accuracy: 0.3667
Epoch 101/200
2/2 [==============================] - 2s 903ms/step - loss: 0.6890 - categorical_accuracy: 0.9851 - val_loss: 2.6398 - val_categorical_accuracy: 0.6000
Epoch 102/200
2/2 [==============================] - 2s 895ms/step - loss: 0.6698 - categorical_accuracy: 0.9943 - val_loss: 2.6864 - val_categorical_accuracy: 0.1667
Epoch 103/200
2/2 [==============================] - 2s 881ms/step - loss: 0.6553 - categorical_accuracy: 0.9977 - val_loss: 2.6761 - val_categorical_accuracy: 0.5333
Epoch 104/200
2/2 [==============================] - 2s 906ms/step - loss: 0.6489 - categorical_accuracy: 1.0000 - val_loss: 2.6830 - val_categorical_accuracy: 0.5333
Epoch 105/20

2/2 [==============================] - 2s 955ms/step - loss: 0.3103 - categorical_accuracy: 1.0000 - val_loss: 1.4022 - val_categorical_accuracy: 0.7667
Epoch 198/200
2/2 [==============================] - 2s 876ms/step - loss: 0.3133 - categorical_accuracy: 1.0000 - val_loss: 1.1763 - val_categorical_accuracy: 1.0000
Epoch 199/200
2/2 [==============================] - 2s 897ms/step - loss: 0.3138 - categorical_accuracy: 0.9988 - val_loss: 1.3128 - val_categorical_accuracy: 0.9000
Epoch 200/200
2/2 [==============================] - 2s 895ms/step - loss: 0.3068 - categorical_accuracy: 1.0000 - val_loss: 1.2006 - val_categorical_accuracy: 0.9333


In [ ]:
model.summary()

 -  결과보기 
 - visualize the result of model



 cmd 창을 킨다 <br>
open cmd  <br>
<br>
C:\Users\User>cd Logs <br>
<br>
C:\Users\User\Logs>cd Train<br>
<br>
C:\Users\User\Logs\train>dir<br>

- 파일을 텐서플로우로 실행시킨다. (run the file )
C:\Users\User\Logs\train>tensorboard --logdir=. 

- 링크를 복사하여 크롬창에 띄운다 ( copy the like and open it at Chrome)

# 예측하기 
8. Make Predictions

In [18]:
res = model.predict(X_train)
print(res[0])
#print(res[1])
#print(res[2])

[9.5860494e-05 1.0751199e-05 3.2533037e-05 9.9939680e-01 2.7878537e-05
 8.3947373e-07 4.6629295e-05 1.6483340e-05 2.8199695e-06 3.6948000e-04]


In [469]:
list_a['한국어'][np.argmax(res[5])]

'학생'

In [25]:
actions[np.argmax(res[3])]

'고혈압'

In [26]:
actions[np.argmax(y_test[3])]

'수술'

In [394]:
print(res)

[[4.9640068e-18 0.0000000e+00 1.5919043e-16 1.4707104e-31 1.8099351e-25
  1.0000000e+00]
 [4.0188888e-17 0.0000000e+00 2.1736209e-14 9.0445225e-31 3.3831052e-26
  1.0000000e+00]
 [1.6919169e-18 0.0000000e+00 1.4745022e-16 9.0753119e-32 1.1275458e-25
  1.0000000e+00]
 [6.6560445e-18 0.0000000e+00 7.9183197e-16 1.9832137e-31 7.8578281e-26
  1.0000000e+00]
 [2.6333616e-16 0.0000000e+00 3.2206842e-15 5.1600234e-31 2.5809288e-25
  1.0000000e+00]
 [9.1876541e-16 0.0000000e+00 6.7831108e-16 1.1555105e-31 8.1771823e-25
  1.0000000e+00]
 [3.8189402e-16 0.0000000e+00 3.8108072e-15 5.8618123e-31 3.0372979e-25
  1.0000000e+00]
 [7.1778449e-18 0.0000000e+00 1.3950404e-16 7.0537231e-31 4.3056709e-25
  1.0000000e+00]
 [8.5220481e-18 0.0000000e+00 1.1018443e-14 4.8758796e-31 2.1577212e-26
  1.0000000e+00]
 [2.5795515e-17 0.0000000e+00 2.7959687e-16 1.9994851e-31 3.8039326e-25
  1.0000000e+00]
 [5.4649117e-17 0.0000000e+00 5.0526732e-14 6.3965701e-31 2.1879997e-26
  1.0000000e+00]
 [5.7125589e-17 0.000

In [464]:
list_a['한국어'][np.argmax(y_test[0])]

'화장실'

# 모델 저장 
9. Save Weights

In [28]:
model.save('action9.h5')

In [338]:
model.load_weights('action.h5')

# 모델 평가 
10. Evaluation using Confusion Matrix and Accuracy

In [29]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [30]:
yhat = model.predict(X_train)

In [31]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [32]:
multilabel_confusion_matrix(ytrue, yhat)
# 이원교차표 출력 

array([[[512,   0],
        [  0,  58]],

       [[510,   0],
        [  0,  60]],

       [[516,   0],
        [  0,  54]],

       [[511,   0],
        [  0,  59]],

       [[512,   0],
        [  0,  58]],

       [[515,   0],
        [  0,  55]],

       [[514,   0],
        [  0,  56]],

       [[514,   0],
        [  0,  56]],

       [[514,   0],
        [  0,  56]],

       [[512,   0],
        [  0,  58]]], dtype=int64)

In [33]:
accuracy_score(ytrue, yhat)

1.0

# 실시간 예측하기 
11. Test in Real Time

In [34]:
# 동영상에 점 찍기 

def draw_landmarks(image, results):  # 동영상의 frame과 동영상에서 받은 result를 받는다. 
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # 얼굴의 점을 연결하여 그린다.
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # POSE를 연결햐여 그린다.
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # 왼손 그리기
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # 오른손 그리기 

In [35]:
# 더 정확한 판단을 위해 각각의 노드에 두개의 점을 추가한다. 
#add aditional 2 spots to the image

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),  
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [36]:
from scipy import stats

In [37]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [38]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

NameError: name 'image' is not defined

<Figure size 1296x1296 with 0 Axes>

In [41]:
from tensorflow.keras.models import load_model

model = load_model('action9.h5')


from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from scipy import stats
import cv2


actions = np.array(['고치다','멈추다','춥다','고혈압', '저혈압','수술' ,'피곤하다','x-ray사진','충격','약'])


def cv2_draw_label(image, text, point):
    x, y = point
    x, y = int(x), int(y)
    pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    font = 'malgun.ttf' 
    draw = ImageDraw.Draw(pil_image)
    try:
        imageFont = ImageFont.truetype(font, 30) #글자크기 
    except:
        imageFont = ImageFont.load_default()
    draw.text((x, y), text, font=imageFont, fill=(255,255,255))  # 글자 색깔 
    image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)

    return image



# 내가 고찬거 
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-40:] #마지막 30프레임
        
        if len(sequence) == 40: #30개차면 predict한다 
            res = model.predict(np.expand_dims(sequence, axis=0))[0] # 2차원을 3차원으로 늘려준다. [0]
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
   

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            #image = prob_viz(res, actions, image, colors)
        print(sentence)    
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        image = cv2_draw_label(image, ' '.join(sentence), (0,10))
        #cv2.putText(image, ' '.join('hello'), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'mp_holistic'

In [ ]:
# 1. New detection variables #30개 차면 detect 할 수 있게
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:] #마지막 30프레임
        
        if len(sequence) == 30: #30개차면 predict한다 
            res = model.predict(np.expand_dims(sequence, axis=0))[0] # 2차원을 3차원으로 늘려준다. [0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()